# Método dos Elementos Finitos - Trabalho 2

Este trabalho visa aplicar o MEF a uma estrutura de treliças.

Para tanto, deverá ler um arquivo em disco com os dados da treliça para resolver via código em Python.

Após ler o arquivo, o código abaixo gera um objeto de classe própria com as propriedades do sistema contido no arquivo.

A seguir o arquivo lido e suas principais características.

In [1]:
%display latex
import functions as fct
#from sage.symbolic.integration.integral import definite_integral,indefinite_integral

In [2]:
tr = fct.trelica("trelica.json")
print('Sistema de {} nós'.format(tr.n))
print('A interligação entre os nós pode ser representada pela seguinte matriz:\n{}\nOnde os elementos fora da diagonal principal representam os valores de 1/EA da barra entre os nós respectivos.'.format(tr.barras))
print('A matriz abaixo representa as cargas no nó:\n{}\nOnde a linha representa o nó e a coluna as dimensões x, y e z, respectivamente.'.format(tr.cargas))
print('Abaixo, temos as seguintes configurações para os nós (coordenada x,y,z e se há apoio na direção):')
for n in tr.nos:
    print(n.tag, '({}, {}, {})'.format(n.x, n.y, n.z), '--> [Rx:{}    Ry:{}    Rz:{}]'.format(n.Rx, n.Ry, n.Rz))

Sistema de 3 nós
A interligação entre os nós pode ser representada pela seguinte matriz:
[[0.    0.001 0.001]
 [0.001 0.    0.001]
 [0.001 0.001 0.   ]]
Onde os elementos fora da diagonal principal representam os valores de 1/EA da barra entre os nós respectivos.
A matriz abaixo representa as cargas no nó:
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 1. -1.  0.]]
Onde a linha representa o nó e a coluna as dimensões x, y e z, respectivamente.
Abaixo, temos as seguintes configurações para os nós (coordenada x,y,z e se há apoio na direção):
Nó 1 (0.0, 0.0, 0.0) --> [Rx:True    Ry:True    Rz:False]
Nó 2 (1.0, 0.0, 0.0) --> [Rx:False    Ry:True    Rz:False]
Nó 3 (1.0, 1.0, 0.0) --> [Rx:False    Ry:False    Rz:False]


A partir deste sistema, temos as seguintes equações difenrenciais para o deslocamento $u$ da barra, considerando a aplicação de uma força Normal $F$:

In [3]:
# Declaração das variáveis independentes
x = var('x') # Variável independente (cumprimento)
var('F,N,E,A,L,i,j') # Variáveis simbólicas de apoio
# Considerações e Restrições das variáveis de apoio
assume(N,i,j,'integer') # Número inteiro de elementos
assume(L>0) # Comprimento da barra positivo
assume(N>0) # Pelo menos 1 Elemento
assume(E>0) # Módulo de elasticidade do material positivo
assume(A>0) # Área da seção transversal positiva
print(assumptions()) # Exibe um resumo das restrições asumidas até aqui
# Variáveis dependentes
u = function('u')(x) # Função analítica desconhecida u(x)
# Equações Diferenciais de u(x)
eq1 = E*A*diff(u,x,2) == 0
eq2 = E*A*diff(u,x) == F
eq1.show() # Exibe a equação diferencial de primeira ordem de u(x)
eq2.show() # Exibe a equação diferencial de segunda ordem de u(x)

[N is integer, i is integer, j is integer, L > 0, N > 0, E > 0, A > 0]


A*E*diff(u(x), x, x) == 0

A*E*diff(u(x), x) == F

A solução analítica destas equações, dadas abaixo, convergem para a equação conhecida da deformação linear na barra, que é:

\begin{equation*}
    u\left(x\right) = \frac{F}{EA}x
\end{equation*}

In [4]:
sol1 = u == desolve(eq1,u,ivar=x)
sol2 = u == desolve(eq2,u,ivar=x)
sol1.show() # Solução da EDO de 1 Ordem
sol2.show() # Solução da EDO de 2 Ordem

u(x) == _K2*x + _K1

u(x) == _C + F*x/(A*E)

A SRP - Sentença de Resíduos Ponderados, fornece a seguinte integral:

\begin{equation}
    \int_{D}{\phi_{i}R}dD = 0
\end{equation}

Onde,

\begin{equation*}
    R = \left(EA\frac{d^{2}\bar{u}}{dx^{2}}\right)
\end{equation*}

e,

\begin{equation*}
    \bar{u} = \sum_{n=1}^{N+1}u_{i}\phi_{i}
\end{equation*}

Assim, temos

\begin{equation*}
    \int_{0}^{L}{\phi_{i}\left(EA\frac{d^{2}\bar{u}}{dx^{2}}\right)}dx = 0
\end{equation*}


In [5]:
phi_i = function('phi_i')(x,i) # phi_i(x) da SRP
u_i = function('u_i')(i) # u_i
u_b = sum(u_i*phi_i,i,1,N+1) # û(x) estimado
R = (E*A*u_b.diff(x,2)).full_simplify() # Resíduo
u_b.show() # Exibe û
R.show() # Exibe o resíduo
SRP = (phi_i*R).integrate(x,0,L) # Sentença dos Resíduos Ponderados
SRP.show() # Exibe a SRP

sum(phi_i(x, i)*u_i(i), i, 1, N + 1)

(A*E*N + A*E)*u_i(i)*diff(phi_i(x, i), x, x)

(A*E*N + A*E)*integrate(phi_i(x, i)*diff(phi_i(x, i), x, x), x, 0, L)*u_i(i)

Deixando a solução analítica de lado, vamos à implementação

# Barra horizontal

## Matriz de Rigidez do Elemento

A matriz de Rigidez de uma barra dentro da treliça é dada por:

\begin{equation}
    K_{ij} = EA \int_{0}^{L}{\left(\frac{dN_{i}}{dx}\frac{dN_{j}}{dx}\right)}dx
\end{equation}

## Vetor de Cargas Nodais do Elemento

\begin{equation}
    f_{i} = EA \left[N_{i}\frac{d\bar{u}}{dx}\right]_{0}^{L}
\end{equation}

Tem-se que:

\begin{equation}
    \left\{\begin{matrix}
        \phi_{1}\left(x\right) & = & \frac{L-x}{x} \\
        & & \\
        \phi_{2}\left(x\right) & = & \frac{x}{L}
    \end{matrix}\right.
\end{equation}

Logo, Simplificando tudo,

\begin{equation*}
    K_{11} = K_{22} = \frac{EA}{L}
\end{equation*}

\begin{equation*}
    K_{12} = K_{21} = -\frac{EA}{L}
\end{equation*}

Ou,

\begin{equation*}
    \mathbf{K}_{local} =
    \left[\begin{matrix}
        \frac{EA}{L} & -\frac{EA}{L} \\
        -\frac{EA}{L} & \frac{EA}{L}
    \end{matrix}\right]
\end{equation*}

Ou, ainda,

\begin{equation}
    \mathbf{K}_{local} = \frac{EA}{L}
    \left[\begin{matrix}
        1 & -1 \\
        -1 & 1
    \end{matrix}\right]
\end{equation}

E,

\begin{equation*}
    \vec{f}_{local} = F
    \left[\begin{matrix}
        -1 \\
        1
    \end{matrix}\right]
\end{equation*}


# Caso Geral - Barra em Qualquer Direção

\begin{equation*}
    N_{1} = \frac{h_{e}-x_{e}}{h_{e}}
\end{equation*}

\begin{equation*}
    N_{2} = \frac{x_{e}}{h_{e}}
\end{equation*}

\begin{equation*}
    N_{G} =
    \left[\begin{matrix}
        N_{1}cos{\alpha} & N_{1}sin{\alpha} & N_{2}cos{\alpha} & N_{2}sin{\alpha}
    \end{matrix}\right]
\end{equation*}

Logo,

\begin{equation*}
    \frac{\partial N}{\partial x} =
    \left[\begin{matrix}
        \frac{\partial N_{1}}{\partial x}cos{\alpha} & \frac{\partial N_{1}}{\partial x}sin{\alpha} & \frac{\partial N_{2}}{\partial x}cos{\alpha} & \frac{\partial N_{2}}{\partial x}sin{\alpha}
    \end{matrix}\right]
\end{equation*}

\begin{equation*}
    \frac{\partial N}{\partial x} =
    \left[\begin{matrix}
    -\frac{1}{h_{e}}cos{\alpha} & -\frac{1}{h_{e}}sin{\alpha} & \frac{1}{h_{e}}cos{\alpha} & \frac{1}{h_{e}}sin{\alpha}
    \end{matrix}\right]
\end{equation*}

\begin{equation*}
    \frac{\partial N}{\partial x} = \frac{1}{h_{e}}
    \left[\begin{matrix}
        -cos{\alpha} & -sin{\alpha} & cos{\alpha} & sin{\alpha}
    \end{matrix}\right]
    = \left[\mathbf{B}\right]
\end{equation*}

Bem como,

\begin{equation*}
    \left[k_{e}\right] =
    \int_{0}^{h_{e}}{EA\frac{\partial N_{i}}{\partial x}\frac{\partial N_{j}}{\partial x}}dx
\end{equation*}

Ou seja,

\begin{equation*}
    \left[k_{e}\right] =
    \int_{0}^{h_{e}}{\left[\mathbf{B^{T}}\right]EA\left[\mathbf{B}\right]}dx
\end{equation*}

Portanto,

\begin{equation*}
    \left[k_{e}\right] = \frac{EA}{h_{e}^{2}}
    \left[\begin{matrix}
        cos^{2}{(\alpha)} & sin{(\alpha)}cos{(\alpha)} & -cos^{2}{(\alpha)} & -sin{(\alpha)}cos{(\alpha)} \\
        sin{(\alpha)}cos{(\alpha)} & sin^{2}{(\alpha)} & -sin{(\alpha)}cos{(\alpha)} & -sin^{2}{(\alpha)} \\
        -cos^{2}{(\alpha)} & -sin{(\alpha)}cos{(\alpha)} & cos^{2}{(\alpha)} & sin{(\alpha)}cos{(\alpha)} \\
        -sin{(\alpha)}cos{(\alpha)} & -sin^{2}{(\alpha)} & sin{(\alpha)}cos{(\alpha)} & sin^{2}{(\alpha)}
    \end{matrix}\right]
    \int_{0}^{h_{e}}{}dx
    = \frac{EA}{h_{e}^{2}}
    \left[\mathbf{T}\right]
    \int_{0}^{h_{e}}{}dx
\end{equation*}

Onde $\mathbf{T}$ é a Matriz de Transformação ou de Rotação.

Logo,

\begin{equation*}
    \left[k_{e}\right] = \frac{EA}{h_{e}^{2}}
    \left[\mathbf{T}\right]
    \int_{0}^{h_{e}}{}dx =
    \left[
        \frac{EA}{h_{e}^{2}}\left[\mathbf{T}\right]x
    \right]_{0}^{h_{e}} =
    \frac{EA}{h_{e}^{2}}\left[\mathbf{T}\right]h_{e}   
\end{equation*}

Ou,

\begin{equation*}
    \left[k_{e}\right] = \frac{EA}{h_{e}}\left[\mathbf{T}\right]
\end{equation*}


In [6]:
no1 = fct.node(0,0,0)
no2 = fct.node(1,0,0)
barra = fct.bar(no1,no2,1000)
fct.mat_T(barra.a)

array([[ 1.,  0., -1., -0.],
       [ 0.,  0., -0., -0.],
       [-1., -0.,  1.,  0.],
       [-0., -0.,  0.,  0.]])